In [7]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime
import time

## Finding, Recording and Plotting the ISS's trajectory
##### This Jupyter Notebook file serves for data collection and visualisation. First, we request the current position of the ISS using an API and plot it onto a map. In a second step the route the ISS takes over Earth is respeatedly recorded by a 60 seconds interval. The data is saved to a csv-file and also plotted.

In [13]:
# Reading in json data from API
url = 'http://api.open-notify.org/iss-now.json'
currentposition_df = pd.read_json(url)

In [14]:
# Wranging the data into the needed format
currentposition_df['latitude'] = currentposition_df.loc['latitude', 'iss_position']
currentposition_df['longitude'] = currentposition_df.loc['longitude', 'iss_position']
currentposition_df.reset_index(inplace=True)
currentposition_df = currentposition_df.drop(['index', 'message', 'iss_position'], axis = 1)
currentposition_df.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)
currentposition_df

,timestamp,latitude,longitude
0,2022-04-11 22:49:20,-46.7693,64.7927


In [19]:
# Plotting the ISS's current position
import plotly.io as pio
pio.renderers.default='iframe'

fig = px.scatter_geo(currentposition_df, lat='latitude', lon='longitude')
fig.show()

In [11]:
# Recording the trajectory using a loop
record_data = True # needs to be manually switched on (True)
if record_data == True:
    starttime=time.time() # starting a timer (new position will be added every 60 seconds)
    space_station_data_df = pd.DataFrame(columns=['timestamp',	'latitude',	'longitude']) # creating an empty df for our observations (though a list would be more efficient)

    while len(space_station_data_df) <= 90: # as long as we haven't recorded 90 instances, do this loop
        url = 'http://api.open-notify.org/iss-now.json'
        furtherposition_df = pd.read_json(url)
        furtherposition_df['latitude'] = furtherposition_df.loc['latitude', 'iss_position']
        furtherposition_df['longitude'] = furtherposition_df.loc['longitude', 'iss_position']
        furtherposition_df.reset_index(inplace=True)
        furtherposition_df = furtherposition_df.drop(['index', 'message', 'iss_position'], axis = 1)
        furtherposition_df.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)

        space_station_data_df = pd.concat([space_station_data_df, furtherposition_df]) # add newest obversation to the previous ones
        print(space_station_data_df.iloc[-1]) # print last line to make the process visible

        time.sleep(60.0 - ((time.time() - starttime) % 60.0)) # adding a 60 seconds break before entering the while loop anew

timestamp    2022-04-11 20:59:33
latitude                 -34.413
longitude                 2.9362
Name: 0, dtype: object
timestamp    2022-04-11 21:00:33
latitude                -36.9134
longitude                 6.3313
Name: 0, dtype: object
timestamp    2022-04-11 21:01:33
latitude                -39.2942
longitude                 9.9669
Name: 0, dtype: object
timestamp    2022-04-11 21:02:33
latitude                -41.5185
longitude                13.8343
Name: 0, dtype: object
timestamp    2022-04-11 21:03:33
latitude                -43.6022
longitude                  18.02
Name: 0, dtype: object
timestamp    2022-04-11 21:04:33
latitude                -45.5019
longitude                22.5093
Name: 0, dtype: object
timestamp    2022-04-11 21:05:33
latitude                -47.1921
longitude                27.3108
Name: 0, dtype: object
timestamp    2022-04-11 21:06:33
latitude                 -48.646
longitude                32.4209
Name: 0, dtype: object
timestamp    2022-04-11 

In [12]:
# saving df to file after the process of collecting data has finished
space_station_data_df.to_csv('ISS_locations.csv', index=None)